## Running the university population network model

This notebook demos running the university network model and demonstrates a selection of the variables that may be altered.  

We start by activating the project environment and loading the packages required to execute the upcoming code. 
Relevant julia doucmentation: [Package manager](https://docs.julialang.org/en/v1/stdlib/Pkg/), [environments](https://pkgdocs.julialang.org/v1/environments/).


In [ ]:
#Load environment
using Pkg
Pkg.activate("../")

#Required packages
using MAT, Distributions
using LinearAlgebra, Random, DelimitedFiles, Parameters

We next include supporting files that contain functions that will be called when performing simulations.

In [ ]:
"""
Load supporting files
"""
# Files containing other functions needed to run the model
include("include_files_uni_model/parametertypes.jl")
include("include_files_uni_model/contact_tracing_fns.jl")
include("include_files_uni_model/network_generation_fns.jl")
include("include_files_uni_model/additional_fns.jl")
include("include_files_uni_model/intervention_condition_affect_fns.jl")
include("include_files_uni_model/mass_testing_fns.jl")
include("include_files_uni_model/seed_initial_states_fns.jl")
include("include_files_uni_model/student_travel_fns.jl")
include("include_files_uni_model/main_function.jl")

Set the batch of configurations to be run. Options include:

* "run_one_run_baseline_nointerv"
* "run_one_run_baseline"
* "sweep_adherence"
* "alter_test_result_delay"
* "change_teaching_space_transmission"
* "amount_backwards_CT"
* "class_CT_threshold"
* "CT_engagement"
* "asymp_scen"
* "change_scaling"
* "vary_household_allocation_strat"
* "vary_accom_level_lockdown"
* "run_mass_testing", 
* "run_mass_testing_6configs"
* "run_no_interventions", 
* "run_one_run_baseline_nointerv"
* "change_recov_propn"
* "rehouse_strat_check"

In [ ]:
runset = "run_one_run_baseline_nointerv"


Specify the simulation parameters

In [ ]:
countfinal=10   # Number of simulations to be performed per scenario
endtime=77    # Timesteps for each individual simulation

n_students = 25000  # Number of students (7155 on campus, 27278 overall in 2018/2019.)
                    # Estimate 25,000 students in local region (~24,500 based on 2019/2020 records)
n_students_on_campus = 7155
n_cohorts = 84

seed_initial_states_fn = seed_states_using_ODEmodel

RNGseed = 100 # Seed the random number generator

Certain aspects have multiple premade functions. Generating society membership, class assignment and household transmission risk are given below, with alternative function choices given in the file "network_generation_fns.jl"

In [ ]:
"""
Specify society assignment function
"""
# If needed, set a different fn from the default
assign_societies_fn_alt = assign_societies_from_aggregated_data

In [ ]:
"""
Specify function to allocate students to department/cohort & classes
"""
# If needed, set a different class assignment function from the default
generate_classes_fn_alt = generate_classes_all_students 

In [ ]:
"""
Specify household transmission risk by group
"""
# Specify function to perform household group allocation
assign_household_transrisk_fn_alt = assign_household_transmit_household_size!


Load the parameters specified for the chosen scenario (parameters listed in "additional_fns.jl"). Here we only load the first configuration.

In [ ]:
    adherence_config, sameday_config, ton_config, toff_config, work_perc_config,
    num_config, work_or_study_group_CT_threshold_config, prob_backwards_CT_config,
    infector_engage_with_CT_prob_config, CT_engagement_config, probasymp_config,
    transasymp_config, scaling_config, suscep_config,
    n_students_config, recov_propn_config,
    rehouse_strat_config,
    CT_delay_until_test_result_config,
    intervention_fn_config, assign_household_fn_config,
    mass_testing_config = load_configs(runset,countfinal,n_cohorts,n_students)

In [ ]:
it = 1

        adherence = adherence_config[it]
        sameday = sameday_config[it]
        ton = ton_config[it]
        toff = toff_config[it]
        attendence_propns = work_perc_config[it,:,:]
        work_or_study_group_CT_threshold = work_or_study_group_CT_threshold_config[it]
        prob_backwards_CT = prob_backwards_CT_config[it]
        infector_engage_with_CT_prob = infector_engage_with_CT_prob_config[it]
        CT_engagement = CT_engagement_config[it]
        prob_asymp = probasymp_config[it]
        asymp_trans_scaling = transasymp_config[it]
        scaling = scaling_config[it]
        suscep_scaling = suscep_config[it]
        local n_students = n_students_config[it]
        recov_propn = recov_propn_config[it]
        rehouse_strat_active = rehouse_strat_config[it]
        # RNGseed = RNGseed_config[it]
        CT_delay_until_test_result_pmf_alt = CT_delay_until_test_result_config[it,:]
        intervention_fns_alt = intervention_fn_config[it,:]
        generate_student_households_fn_alt = assign_household_fn_config[it]
        mass_testing_parameters_alt = mass_testing_config[it]

Set whether the following interventions are active, based on configuration being run:

* forward contact tracing
* backward contact tracing
* class closures
* mass testing

In [ ]:
# Set if contact tracing is active or not (Bool type variable)
contact_tracing_active = false

# Set if class closures is active or not (Bool type variable)
work_study_group_closure_active = false

# set if backwards contact tracing is active or not (Bool type variable)
perform_CT_from_infector = false


    # Specify if mass testing module is in use
    if (runset == "run_mass_testing") || (runset == "run_mass_testing_6configs") ||
         (runset == "run_mass_testing_alter_adherence") || (runset == "run_mass_testing_frequent")
        mass_testing_active = true
    else
        mass_testing_active = false
    end

        if runset=="amount_backwards_CT"
            perform_CT_from_infector = true
            contact_tracing_active = true
        end

        if runset=="class_CT_threshold"
            work_study_group_closure_active = true
        end

        if (runset=="CT_engagement")||(runset=="amount_backwards_CT")||
            (runset=="sweep_adherence")||(runset=="sweep_adherence_with_rehouse")||
            (runset=="alter_test_result_delay")||
            (runset=="change_teaching_space_transmission")||
            (runset=="vary_accom_level_lockdown")||
            (runset=="run_one_run_accom_lockdown_baseline") ||
            (runset=="vary_household_allocation_strat") ||
            (runset=="run_mass_testing") ||
            (runset=="run_mass_testing_6configs") ||
            (runset=="run_mass_testing_alter_adherence") ||
            (runset=="run_mass_testing_frequent") ||
            (runset=="run_baseline_mass_testing") ||
            (runset=="run_one_run_baseline") ||
            (runset=="rehouse_strat_check")
            contact_tracing_active = true
        end

        # Set society & sports club transmission risk.
        if (runset=="run_no_interventions") || (runset=="run_one_run_baseline_nointerv") ||
            (runset=="trans_risk_scale_no_interv") || (runset=="probasymp_scale_no_interv") ||
            (runset=="transasymp_scale_no_interv")
            # No interventions scenario.
            # Society risk matches social contacts. Sports club matches household risk.
            society_sports_transrisk_mean = [0.2414,0.34]
        else
            # Transmission risk with interventions in place
            society_sports_transrisk_mean = [0.12, 0.2414]
        end

Set up network parameters. Uses requested number of cohort types.

* For options, the function resides in "include_files_uni_model/additional_fns.jl"

In [ ]:
        
        network_parameters, team_generation_parameters = find_network_parameters(n_cohorts,
                                                                                attendence_propns=attendence_propns,
                                                                                n_students = n_students,
                                                                                n_students_on_campus = n_students_on_campus
                                                                                )

        CT_parameters = CT_params(prob_backwards_CT = prob_backwards_CT,
                                    perform_CT_from_infector = perform_CT_from_infector,
                                    infector_engage_with_CT_prob = infector_engage_with_CT_prob,
                                    work_or_study_group_CT_threshold = work_or_study_group_CT_threshold,
                                    CT_engagement = CT_engagement,
                                    CT_delay_until_test_result_pmf = CT_delay_until_test_result_pmf_alt
                                    #test_false_negative_vec = test_false_negative_vec_alt
                                    )

        infection_parameters = infection_params(n_cohorts = n_cohorts,
                                                transrisk_household_18_34_group_mean=scaling,
                                                suscep_scaling = suscep_scaling,
                                                probasymp_dist = prob_asymp,
                                                asymp_trans_scaling_dist = asymp_trans_scaling,
                                                adherence = adherence,
                                                recov_propn = recov_propn,
                                                society_sports_transrisk_mean = society_sports_transrisk_mean,
                                                #delay_adherence_pmf = delay_adherence_pmf_alt
                                                #transrisk_household_group = transrisk_household_group_alt
                                                )

        # Establish society generation parameters
        society_generation_parameters = society_generation_params()

Call function (located in "include_files_network_model/main_function.jl") and unpack the output variable.

In [ ]:
 if contact_tracing_active==true
                @time  output = uni_network_run(RNGseed,
                                                n_students,
                                                ton,toff,
                                                infection_parameters,
                                                sameday,
                                                seed_initial_states_fn,
                                                countfinal,
                                                endtime,
                                                contact_tracing_active,
                                                CT_parameters,
                                                network_parameters,
                                                team_generation_parameters,
                                                society_generation_parameters,
                                                work_study_group_closure_active,
                                                mass_testing_active,
                                                rehouse_strat_active,
                                                mass_testing_parameters = mass_testing_parameters_alt,
                                                intervention_fns = intervention_fns_alt,
                                                generate_student_households_fn = generate_student_households_fn_alt,
                                                assign_societies_fn = assign_societies_fn_alt,
                                                generate_classes_fn = generate_classes_fn_alt,
                                                assign_household_transrisk_fn = assign_household_transrisk_fn_alt
                                                )
                @unpack numlat, numinf, numrep,
                        prevlat, prevsymp, prevasymp, prevpresymp, prevrec,
                        newinf, newasymp, atworkinf, atworkasymp, infected_by,
                        num_isolating, num_isolating_oncampus, num_isolating_offcampus,
                        num_household_isolating, num_symp_isolating, num_asymp_isolating,
                        num_isolating_CTcause,
                        num_accom_lockdown_isolating,
                        num_CT, num_infected, social_dynamic_infection_count,
                        accommodation_dynamic_infection_count, household_infection_count,
                        var_num_infected, num_init_infected, mean_init_generation_time,
                        Rt,
                        new_rehoused, current_rehoused,
                        tests_performed, test_outcomes,
                        cohort_infection_count, society_infection_count,
                        n_oncampus_inf, n_offcampus_inf,
                        n_isol_adhering, n_isol_adhering_oncampus, n_isol_adhering_offcampus = output
else
               @time  output = uni_network_run(RNGseed,
                                               n_students,
                                               ton,toff,
                                               infection_parameters,
                                               sameday,
                                               seed_initial_states_fn,
                                               countfinal,
                                               endtime,
                                               contact_tracing_active,
                                               CT_parameters,
                                               network_parameters,
                                               team_generation_parameters,
                                               society_generation_parameters,
                                               work_study_group_closure_active,
                                               mass_testing_active,
                                               rehouse_strat_active,
                                               mass_testing_parameters = mass_testing_parameters_alt,
                                               intervention_fns = intervention_fns_alt,
                                               generate_student_households_fn = generate_student_households_fn_alt,
                                               assign_societies_fn = assign_societies_fn_alt,
                                               generate_classes_fn = generate_classes_fn_alt,
                                               assign_household_transrisk_fn = assign_household_transrisk_fn_alt
                                               )
               @unpack  numlat, numinf, numrep,
                        prevlat, prevsymp, prevasymp, prevpresymp, prevrec,
                        newinf, newasymp, atworkinf, atworkasymp, infected_by,
                        num_isolating, num_isolating_oncampus, num_isolating_offcampus,
                        num_household_isolating, num_symp_isolating, num_asymp_isolating,
                       num_accom_lockdown_isolating,
                       num_CT, num_infected,
                       social_dynamic_infection_count,
                       accommodation_dynamic_infection_count, household_infection_count,
                       var_num_infected,
                       num_init_infected, mean_init_generation_time, Rt,
                       new_rehoused, current_rehoused,
                       tests_performed, test_outcomes,
                       cohort_infection_count,
                       society_infection_count,
                       n_oncampus_inf, n_offcampus_inf,
                       n_isol_adhering, n_isol_adhering_oncampus, n_isol_adhering_offcampus = output
 end

The outputs may then undergo further analysis and/or be saved to file.